In [ ]:
!pip install simpletransformers

In [ ]:
!wget  https://www.dropbox.com/s/sukruo2u3tiq5fr/punct-data.zip?dl=1 -O punct-data.zip
!unzip punct-data.zip

--2022-05-20 21:46:01--  https://www.dropbox.com/s/sukruo2u3tiq5fr/punct-data.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/sukruo2u3tiq5fr/punct-data.zip [following]
--2022-05-20 21:46:01--  https://www.dropbox.com/s/dl/sukruo2u3tiq5fr/punct-data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf1a5392ee7db06880c3908e1ec.dl.dropboxusercontent.com/cd/0/get/Blqcwg9cODOnDcRD4NP-edOUx5HikjT3QD4DeBhvz9Xsvlr14C-QwVCdfjdH5EoYXhwFxSDnP5DsyvlyQ-gcxDSjQM7dcg2LZvPV1Bj1grrIl5tWaeVXvd2lEYx0PHmkRV8gZKGWiJKFOVhxsEKcKERyASXtGw6PDNw45XTjkZvSrGYDAUoRt7JA6m35l5e27VU/file?dl=1# [following]
--2022-05-20 21:46:01--  https://ucf1a5392ee7db06880c3908e1ec.dl.dropboxusercontent.com/cd/0/get/Blqcwg9cODOnDcRD4NP-edOUx5HikjT3QD4DeBh

In [ ]:
import os
import pandas as pd

BASE_DIR = 'punct-data/'

dataset = {
    
}

for filename in os.listdir(BASE_DIR):
  dataset[filename.replace('.csv', '')] = pd.read_csv(os.path.join(BASE_DIR, filename)).dropna()
 

In [ ]:
dataset['train']

,sentence_id,labels,words
0,0,O,Filmado
1,0,O,na
2,0,O,cidade
3,0,O,de
4,0,O,Nova
...,...,...,...
239072,58213,O,democracia
239073,58213,O,direta
239074,58213,O,ter
239075,58213,I-PERIOD,sucesso


In [ ]:
!pip install wandb

import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import wandb

# Create a new run
with wandb.init(project="bert-base-cased-ner") as run:

  # Connect an Artifact to the run
  model_name = "neuralmind/bert-base-portuguese-cased"
  #my_model_artifact = run.use_artifact(my_model_name)

  # Download model weights to a folder and return the path
  #model_dir = my_model_artifact.download()
  train_args = {
      'evaluate_during_training': True,
      'logging_steps': 10,
      'num_train_epochs': 15,
      'evaluate_during_training_steps': 5,
      'save_eval_checkpoints': False,
      'train_batch_size': 16,
      'eval_batch_size': 8,
      'overwrite_output_dir': True,
      'labels_list':dataset['train'].labels.unique().tolist(),
      'use_early_stopping':True
  }


  model = NERModel(
      "bert",
      model_name,
      args=train_args,
      use_cuda=torch.cuda.is_available()
  )
  model.train_model(dataset['train'], eval_data=dataset['val'])
  result, model_outputs, wrong_preds = model.eval_model(dataset['val'])

  pd.DataFrame.from_dict(result, orient='index').T.to_csv('test_result.csv', index=False,index_label=False)


In [ ]:
path_to_dataset = '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/redacoes_mec_transcritas_atualizado.csv'

students_df = pd.read_csv(path_to_dataset)
students_df.head()

In [ ]:
import re
students_texts = students_df.texto.apply(lambda text: re.sub(r'[.,?]', '', text).lower()).tolist()

In [ ]:
predictions, raw_outputs = model.predict(students_texts)


In [ ]:
new_pred = []
for i,pred in enumerate(predictions, 1):
    for item in pred:
        new_pred.append([i, tuple(item.items())[0][0], tuple(item.items())[0][1]])

In [ ]:
pred_df = pd.DataFrame(new_pred, columns=['sentence_id', 'words', 'labels'])

pred_df.to_csv('pred_df.csv', index=False, index_label=False)

In [ ]:
import spacy
from spacy import displacy

In [ ]:
path_to_dataset = '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/'


In [ ]:
import pandas as pd
import re 

pred_df = pd.read_csv('pred_df.csv')
label2dict = {"COMMA":",","PERIOD": ".", "QUESTION":"?"}
ents = []
for id, group in pred_df.groupby("sentence_id"):
  text = " ".join(group.words)
  #print(text)
  ent = {
            "text":text,
             "ents":[],
             "title": None
   }
  for i, label in enumerate(group.labels):
      #print(group.words)
      if label != "O":
      
        matching_word = group.words.tolist()[i]
        #print(matching_word)
        matching_word = re.sub(r'\(', '\(', matching_word)
        matching_word = re.sub(r'\)', '\)', matching_word)
        
       
        match = re.search(matching_word, text)
        ent["ents"].append({
            "start":match.start(),
            "end":match.end(),
            "label":label
        })

  ents.append(ent)

  #print(" ".join(group.labels))

In [ ]:

displacy.render([ents[2]], style="ent", manual=True, jupyter=True)

In [ ]:
!zip model-15.zip -r outputs/checkpoint-7725-epoch-15/

In [ ]:
!cp model-15.zip '/content/drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/model-15.zip'